In [53]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-139285
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-139285


In [54]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = 'compute-cluster'

# very the cluster is not aleady created
if cpu_cluster_name in ws.compute_targets:
    compute_target = ws.compute_targets[cpu_cluster_name]
    print('Found an existing cluster to use')
else:
    print('Creating a new compute cluster')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

Found an existing cluster to use
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-21T20:43:41.026000+00:00', 'errors': None, 'creationTime': '2021-02-21T19:41:33.839896+00:00', 'modifiedTime': '2021-02-21T19:41:49.328162+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [71]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.5, 1),
        '--max_iter': choice(50, 100, 150)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=os.path.join('training'),
    entry_script='train.py',
    compute_target=compute_target)

# est = ScriptRunConfig(
#     source_directory=os.path.join('./'),
#     script='train.py',
#     compute_target=compute_target
# )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    max_total_runs=20,
    max_concurrent_runs=4
)

In [72]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == 'Completed')

RunId: HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-139285/workspaces/quick-starts-ws-139285

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-21T21:35:12.286873][API][INFO]Experiment created<END>\n""<START>[2021-02-21T21:35:13.010585][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-02-21T21:35:13.7735605Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-02-21T21:35:13.356954][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69?wsid=/subscriptions/1b944

In [73]:
# from azureml.widgets import RunDetails

# RunDetails(hyperdrive_run).show()

In [74]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

best_model = hyperdrive_run.get_best_run_by_primary_metric()

print('Best Model:', best_model.get_metrics())
print('Best Model Details:', best_model.get_details())
print('Best Model File Name:', best_model.get_file_names())

[{'run_id': 'HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69_1', 'hyperparameters': None, 'best_primary_metric': 0.9118866970156804, 'status': 'Completed'}, {'run_id': 'HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69_3', 'hyperparameters': None, 'best_primary_metric': 0.9118866970156804, 'status': 'Completed'}, {'run_id': 'HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69_2', 'hyperparameters': None, 'best_primary_metric': 0.9118866970156804, 'status': 'Completed'}, {'run_id': 'HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69_5', 'hyperparameters': None, 'best_primary_metric': 0.9109762266059687, 'status': 'Completed'}, {'run_id': 'HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69_4', 'hyperparameters': None, 'best_primary_metric': 0.9109762266059687, 'status': 'Completed'}, {'run_id': 'HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69_0', 'hyperparameters': None, 'best_primary_metric': 0.9109762266059687, 'status': 'Completed'}, {'run_id': 'HD_d06835ff-ceaf-42e6-aa1f-82c97cf6db69_preparation', 'hyperparameters': None, 'best_primary_metric

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)
automl_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_model, fitted_model = automl_run.get_output()

print('Best Model:', automl_best_model.get_metrics())
print('Best Model Details:', automl_best_model.get_details())
print('Best Model File Name:', automl_best_model.get_file_names())

print(automl_best_model)

In [ ]:
compute_target.delete()